# 数据规整化：清理、转换、合并、重塑

## 目录
+ 合并数据集
 + 数据库风格的DataFrame合并  
 merge函数
   + 一个键
     + 多对一
     + 多对多
   + 多个键
   + 对重复列名的处理
 + 索引上的合并  
 merge函数，left_index、right_index参数
   + 列对索引
     + 层次化索引
   + 索引对索引
   + join方法
     + 一组DataFrame
 + 轴向连接  
 concat函数
   + NumPy数组的concatenation函数
   + Series
   + DataFrame
 + 合并重叠数据  
 combine_first方法
     + Series
     + DataFrame


+ 重塑和轴向选择
 + 重塑层次化索引
   + stack：将数据的列“选择”为行
   + unstack：将数据的行“选择”为列
 + 将“长格式”旋转为“宽格式”
 等价于set_index创建层次化索引，再用unstack重塑


+ 数据转换
 + 移除重复数据
 + 利用函数或映射进行数据转换  
map方法，实现元素级转换以及其他数据清理工作的便捷方式
 + 替换值
 + 重命名轴索引
   + map方法
   + rename方法
 + 离散化和面元划分
   + cut函数
   + qcut函数
 + 检测和过滤异常值
 + 排列和随机采样
   + 随机重排序
   + 选取随机子集（不重复抽样）
   + 选取样本（重复抽样）
 + 计算指标/哑变量
   + 每行只属于一个分类
   + 每行同属于多个分类
   + 离散化与哑变量的结合


+ 字符串操作
 + 字符串对象方法
 + 正则表达式
   + re模块中的re对象
   + re.compile编译regex对象
      + 模式匹配
      + 替换
      + 拆分
 + pandas中矢量化的字符串函数

In [1]:
from numpy.random import randn
import numpy as np
from pandas import Series, DataFrame
import pandas as pd

## 合并数据集

### 数据库风格的DataFrame合并
列对列连接，DataFrame对象中的索引被丢弃

#### merge函数的参数
+ left  
参与合并的左侧DataFrame
+ right  
参与合并的右侧DataFrame
+ how  
“inner”、“outer”、“left”、“right”其中之一。默认为“inner”
+ on  
用于连接的列名。必须存在于左右两个DataFrame对象中。如果未指定，且其他连接键也未指定，则以left和right列名的交集作为连接键
+ left_on  
左侧DataFrame中用作连接键的列
+ right_on  
右侧DataFrame中用作连接键的列
+ left_index  
将左侧的行索引用作连接键
+ right_index  
类似于left_index
+ sort  
根据连接键对合并后的数据进行排序，默认为True。有时在处理大数据集时，禁用该选项可获得更好的性能
+ suffixes  
字符串值元组，用于追加到重叠列名的末尾，默认为（‘_x’，‘_y’）。例如，如果左右两个DataFrame对象都有“data”，则结果中就会出现“data_x”和“data_y”
+ copy  
设置为False，可以在某些特殊情况下避免将数据复制到结果数据结构中。默认总是复制

#### 一个键

##### 多对一

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df1

In [3]:
df2 = DataFrame(dict(key=['a', 'b', 'd'], data2=range(3)))
df2

In [4]:
pd.merge(df1, df2) # 如果没有指定，merge就会将重叠列的列名当做键

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
pd.merge(df1, df2, on='key') # 显式指定按哪个列连接

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [6]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [7]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [8]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey') # 两个对象的列名不同，可以分别指定左键、右键

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
pd.merge(df1, df2, how='outer') # 默认内连接“inner”，结果中的键是交集；其他方式还有“left”、“right”、“outer”

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


##### 多对多

In [10]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [11]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [12]:
pd.merge(df1, df2, on='key', how='left') # 多对多连接产生的是行的笛卡尔积

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


#### 多个键

In [13]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [14]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [15]:
pd.merge(left, right, on=['key1', 'key2'], how='outer') # 多个键进行合并，传入一个由列名组成的列表；多对多，笛卡尔积

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


#### 对重复列名的处理
suffixes选项，用于指定附加到左右两个DataFrame对象的重叠列名上的字符串

In [16]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引上的合并

#### 列对索引
传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键

In [17]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [18]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [19]:
pd.merge(left1, right1, left_on='key', right_index=True) # 左侧DataFrame的列对右侧DataFrame的键

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [20]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # 默认内连接“inner”，通过how参数指定外连接

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


##### 层次化索引
以列表的形式指明用作合并键的多个列

In [21]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [22]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
righth

event1 
 event2 
 
 
 
 
 Nevada 
 2001 
 0 
 1 
 
 
 2000 
 2 
 3 
 
 
 Ohio 
 2000 
 4 
 5 
 
 
 2000 
 6 
 7 
 
 
 2001 
 8 
 9 
 
 
 2002 
 10 
 11

In [23]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True) # left_on列表中的键，对应右侧DataFrame的MultiIndex

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [24]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer') # 默认内连接“inner”，通过how参数指定外连接

,data,key1,key2,event1,event2
0,0.0,Ohio,2000.0,4.0,5.0
0,0.0,Ohio,2000.0,6.0,7.0
1,1.0,Ohio,2001.0,8.0,9.0
2,2.0,Ohio,2002.0,10.0,11.0
3,3.0,Nevada,2001.0,0.0,1.0
4,4.0,Nevada,2002.0,NaN,NaN
4,NaN,Nevada,2000.0,2.0,3.0


#### 索引对索引

In [25]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
                 columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [26]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [27]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


#### join方法
默认按索引连接

In [28]:
left2.join(right2, how='outer') # 默认左连接“left”，通过how参数指定外连接

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [29]:
left1.join(right1, on='key') # on参数，支持参数DataFrame的索引跟调用者DataFrame的某个列之间的连接，left1的‘key’列与right1的索引匹配

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


##### 一组DataFrame

In [30]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [31]:
left2.join([right2, another]) # 默认左连接“left”

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### 轴向连接
concat函数

#### NumPy数组的concatenation函数

In [32]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [33]:
np.concatenate([arr, arr], axis=1) # 默认沿0轴连接，设置axis参数沿1轴连接（即横向连接）

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

#### concat函数的参数
+ objs  
参与连接的pandas对象的列表或字典。唯一必需的参数
+ axis  
指明连接的轴向。默认为0
+ join  
“inner”、“outer”其中之一，默认为“outer”。指明其他轴向上的索引是按交集（inner）还是并集（outer）进行合并
+ join_axes  
指明用于其他n-1条轴的索引，不执行并集/交集运算
+ keys  
与连接对象有关的值，用于形成连接轴向上的层次化索引。可以是任意值的列表或数组、元祖数组、数组列表（如果将levels设置成多级数组的话）
+ levels  
指定用作层次化索引各级别上的索引，如果设置了keys的话
+ names  
用于创建分层级别的名称，如果设置了keys和（或）levels的话
+ verify_integrity  
检查结果对象新轴上的重复情况，如果发现则引发异常。默认（False）允许重复
+ ignore_index  
不保留连接轴上的索引，产生一组新索引range(total_length)

#### Series

In [34]:
s1 = Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [35]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [36]:
s3 = Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [37]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [38]:
pd.concat([s1, s2, s3]) # 默认沿0轴连接（即纵向拼接）

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [39]:
pd.concat([s1, s2, s3], axis=1) # 传入axis=1（即横向连接），结果变成DataFrame

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [40]:
pd.concat([s1, s4], axis=1, join='inner') # 默认外连接，传入join=‘inner’，设置内连接

,0,1
a,0,0
b,1,5


In [41]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']]) # join_axes指定其他轴上使用的索引，此处沿1轴连接，指定结果DataFrame的行索引

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [42]:
pd.concat([s1, s1, s3], keys=['one', 'two', 'three']) # 沿0轴连接时，通过keys参数，在连接轴上创建一个层次化索引

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [43]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three']) # 沿1轴连接时，由于Series没有1轴，设置keys参数，keys会成为DataFrame的列头

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


#### DataFrame

In [44]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [45]:
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [46]:
pd.concat([df1, df2], axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [47]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2']) # keys参数在连接轴上创建层次化索引

level1 
 level2 
 
 
 
 one 
 two 
 three 
 four 
 
 
 
 
 a 
 0 
 1 
 5.0 
 6.0 
 
 
 b 
 2 
 3 
 NaN 
 NaN 
 
 
 c 
 4 
 5 
 7.0 
 8.0

In [48]:
pd.concat({'level1': df1, 'level2': df2}, axis=1) # 传入的不是列表而是字典，则字典的键会被当做keys参数的值，以上两种方式等价

level1 
 level2 
 
 
 
 one 
 two 
 three 
 four 
 
 
 
 
 a 
 0 
 1 
 5.0 
 6.0 
 
 
 b 
 2 
 3 
 NaN 
 NaN 
 
 
 c 
 4 
 5 
 7.0 
 8.0

In [49]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower']) # 给keys创建的层次化索引，赋予names属性

upper 
 level1 
 level2 
 
 
 lower 
 one 
 two 
 three 
 four 
 
 
 
 
 a 
 0 
 1 
 5.0 
 6.0 
 
 
 b 
 2 
 3 
 NaN 
 NaN 
 
 
 c 
 4 
 5 
 7.0 
 8.0

In [50]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,0.323341,1.524945,-2.259796,0.060188
1,-0.588303,2.018010,-1.237038,-0.616089
2,-0.398261,-0.424968,1.234534,-0.978196


In [51]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,-0.418186,-1.062889,-1.720135
1,0.288955,-0.438726,-0.294381


In [52]:
pd.concat([df1, df2], ignore_index=True) # 丢弃连接轴上原有的索引，产生新的数字索引

,a,b,c,d
0,0.323341,1.524945,-2.259796,0.060188
1,-0.588303,2.018010,-1.237038,-0.616089
2,-0.398261,-0.424968,1.234534,-0.978196
3,-1.720135,-0.418186,NaN,-1.062889
4,-0.294381,0.288955,NaN,-0.438726


In [53]:
pd.concat([df1, df2], ignore_index=True, axis=1) # 与上例对比，合并一定是按索引对齐的，ignore_index=True只是重新创建列标签

,0,1,2,3,4,5,6
0,0.323341,1.524945,-2.259796,0.060188,-0.418186,-1.062889,-1.720135
1,-0.588303,2.018010,-1.237038,-0.616089,0.288955,-0.438726,-0.294381
2,-0.398261,-0.424968,1.234534,-0.978196,NaN,NaN,NaN


### 合并重叠数据
combine_first方法

#### Series

In [54]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [55]:
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [56]:
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [57]:
b[:-2].combine_first(a[2:]) # 合并后的结果是b[:-2]与a[2:]的并集，重复值取调用者对象中的值，只有调用者中缺失值才取参数者，以上两种方式等价

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

#### DataFrame

In [58]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [59]:
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [60]:
df1.combine_first(df2) # 合并后的结果是df1和df2的并集，重复值取调用者对象中的值，只有调用者中缺失值才取参数者

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 重塑和轴向选择

### 重塑层次化索引
+ stack  
将数据的列“旋转”为行
+ unstack  
将数据的行“旋转”为列

In [61]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [62]:
result = data.stack() # 将列转换为行，DataFrame转换为层次化索引的Series
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [63]:
result.unstack() # 将行转换为列，层次化索引的Series转换为DataFrame

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [64]:
result.unstack(0) # 默认情况下，stack和unstack操作的是最内层，传入分层级别的编号或名称，对其他级别进行操作

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [65]:
result.unstack('state') # 传入分层级别的名称，以上两种方式等价

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [66]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [67]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
s2

c    4
d    5
e    6
dtype: int64

In [68]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [69]:
data2.unstack() # 如果不是所有的级别值都能在分组中找到，unstack操作会引入缺失值NaN

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [70]:
data2.unstack().stack() # stack默认会滤除缺失数据，因此unstack和stack运算可逆

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [71]:
data2.unstack().stack(dropna=False) # 通过dropna=False，则保留缺失值

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [72]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

side 
 left 
 right 
 
 
 state 
 number 
 
 
 
 
 
 
 Ohio 
 one 
 0 
 5 
 
 
 two 
 1 
 6 
 
 
 three 
 2 
 7 
 
 
 Colorado 
 one 
 3 
 8 
 
 
 two 
 4 
 9 
 
 
 three 
 5 
 10

In [73]:
df.unstack('state')

side 
 left 
 right 
 
 
 state 
 Ohio 
 Colorado 
 Ohio 
 Colorado 
 
 
 number 
 
 
 
 
 
 
 
 
 one 
 0 
 3 
 5 
 8 
 
 
 two 
 1 
 4 
 6 
 9 
 
 
 three 
 2 
 5 
 7 
 10

In [74]:
df.unstack('state').stack('side') # 对DataFrame进行stack和unstack操作时，作为旋转轴的级别会成为结果中的最低级别，被放置在最内层

state 
 Ohio 
 Colorado 
 
 
 number 
 side 
 
 
 
 
 
 
 one 
 left 
 0 
 3 
 
 
 right 
 5 
 8 
 
 
 two 
 left 
 1 
 4 
 
 
 right 
 6 
 9 
 
 
 three 
 left 
 2 
 5 
 
 
 right 
 7 
 10

### 将“长格式”旋转为“宽格式”
pivot方法

In [75]:
data = pd.read_csv('./data/ch07/macrodata.csv')
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))

data.head(10)

item,realgdp,infl,unemp
date,,,
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2
1960-06-30,2834.390,0.14,5.2
1960-09-30,2839.022,2.70,5.6
1960-12-31,2802.616,1.21,6.3
1961-03-31,2819.264,-0.40,6.8


In [76]:
ldata = data.stack().reset_index().rename(columns={0: 'value'}) # 引入“长格式”的时间序列数据
ldata.head(10)

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [77]:
wdata = ldata.pivot('date', 'item', 'value') # 将“长格式”旋转为“宽格式”
wdata.head(10)

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [78]:
pivoted = ldata.pivot('date', 'item', 'value') # 前两个参数值分别是用作行和列索引的列名，最后一个参数值则是用于填充DataFrame的数据列的列名
pivoted.head(10)

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [79]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.821896
1,1959-03-31,infl,0.000,-0.612912
2,1959-03-31,unemp,5.800,-1.146332
3,1959-06-30,realgdp,2778.801,-1.097301
4,1959-06-30,infl,2.340,1.547789
5,1959-06-30,unemp,5.100,0.196852
6,1959-09-30,realgdp,2775.488,0.437029
7,1959-09-30,infl,2.740,1.605312
8,1959-09-30,unemp,5.300,1.676900
9,1959-12-31,realgdp,2785.204,-0.081587


In [80]:
# 有两个需要参与重塑的数据列（value和 value2）
# 如果忽略最后一个参数，得到的DataFrame会带有层次化的列，value在外层，item在内层
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value 
 value2 
 
 
 item 
 infl 
 realgdp 
 unemp 
 infl 
 realgdp 
 unemp 
 
 
 date 
 
 
 
 
 
 
 
 
 
 
 1959-03-31 
 0.00 
 2710.349 
 5.8 
 -0.612912 
 0.821896 
 -1.146332 
 
 
 1959-06-30 
 2.34 
 2778.801 
 5.1 
 1.547789 
 -1.097301 
 0.196852 
 
 
 1959-09-30 
 2.74 
 2775.488 
 5.3 
 1.605312 
 0.437029 
 1.676900 
 
 
 1959-12-31 
 0.27 
 2785.204 
 5.6 
 -0.511770 
 -0.081587 
 -0.659386 
 
 
 1960-03-31 
 2.31 
 2847.699 
 5.2 
 -0.406749 
 0.977615 
 -1.132756

In [81]:
unstacked = ldata.set_index(['date', 'item']).unstack('item') # pivot实际上是个快捷方式：用set_index创建层次化索引，再用unstack重塑
unstacked[:7]

value 
 value2 
 
 
 item 
 infl 
 realgdp 
 unemp 
 infl 
 realgdp 
 unemp 
 
 
 date 
 
 
 
 
 
 
 
 
 
 
 1959-03-31 
 0.00 
 2710.349 
 5.8 
 -0.612912 
 0.821896 
 -1.146332 
 
 
 1959-06-30 
 2.34 
 2778.801 
 5.1 
 1.547789 
 -1.097301 
 0.196852 
 
 
 1959-09-30 
 2.74 
 2775.488 
 5.3 
 1.605312 
 0.437029 
 1.676900 
 
 
 1959-12-31 
 0.27 
 2785.204 
 5.6 
 -0.511770 
 -0.081587 
 -0.659386 
 
 
 1960-03-31 
 2.31 
 2847.699 
 5.2 
 -0.406749 
 0.977615 
 -1.132756 
 
 
 1960-06-30 
 0.14 
 2834.390 
 5.2 
 1.235537 
 0.851554 
 -0.533922 
 
 
 1960-09-30 
 2.70 
 2839.022 
 5.6 
 -1.185304 
 0.445802 
 -0.818831

## 数据转换

### 移除重复数据

In [82]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [83]:
data.duplicated() # duplicated方法返回一个布尔型Series，表示各行与之前的行是否重复

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [84]:
data.drop_duplicates() # drop_duplicates方法，用于返回一个移除了重复行的DataFrame（保留data.duplicated()为False的行）

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [85]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [86]:
data.duplicated(['k1']) # 两种方法默认判断全部列，可以指定部分列进行重复项判断

0    False
1     True
2     True
3    False
4     True
5     True
6     True
dtype: bool

In [87]:
data.drop_duplicates(['k1']) # 两种方法默认判断全部列，可以指定部分列进行重复项判断

,k1,k2,v1
0,one,1,0
3,two,3,3


In [88]:
data.duplicated(['k1', 'k2'], keep='last') # 两种方法默认保留的是第一个出现的值组合，传入keep='last'则保留最后一个

0     True
1    False
2    False
3     True
4    False
5     True
6    False
dtype: bool

In [89]:
data.drop_duplicates(['k1', 'k2'], keep='last') # 两种方法默认保留的是第一个出现的值组合，传入keep='last'则保留最后一个

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### 利用函数或映射进行数据转换
利用Series的map方法，实现元素级转换以及其他数据清理工作的便捷方式

In [90]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


#### 字典

In [91]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [92]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal) # map方法可以接受一个含有映射关系的字典型对象
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


#### 函数

In [93]:
data['food'].map(lambda x: meat_to_animal[x.lower()]) # map方法可以接受一个函数

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 替换值
replace方法

In [94]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [95]:
data.replace(-999, np.nan) # 将-999替换为NaN

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [96]:
data.replace([-999, -1000], np.nan) # 一次性替换多个值，传入一个由待替换值组成的列表以及一个替换值

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [97]:
data.replace([-999, -1000], [np.nan, 0]) # 对不同的值进行不同的替换，传入一个由替换关系组成的列表

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [98]:
data.replace({-999: np.nan, -1000: 0}) # 对不同的值进行不同的替换，也可以传入一个代表映射关系的字典，以上两种方式等价

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

In [99]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


#### map方法

In [100]:
data.index.map(str.upper) # 同Series一样，轴标签也有map方法

array(['OHIO', 'COLORADO', 'NEW YORK'], dtype=object)

In [101]:
data.index = data.index.map(str.upper) # 将其赋值给index，可以对DataFrame进行就地修改
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


#### rename方法

In [102]:
data.rename(index=str.title, columns=str.upper) # 创建修改后的副本（而非就地修改），可以对index和columns传入函数

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [103]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'}) # 也可以对index和columns传入字典

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [104]:
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True) # 通过inplace参数设置就地修改
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 离散化和面元划分
将连续数据离散化或拆分为“面元”（bin）

#### cut函数

In [105]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [106]:
bins = [18, 25, 35, 60, 100]
bins

[18, 25, 35, 60, 100]

In [107]:
cats = pd.cut(ages, bins) # cut函数，第一个参数是待离散化的连续数据，第二个参数是划分的区间
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [108]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [109]:
cats.categories # cut函数返回的Categorical对象，有codes和categories两个属性

Index([u'(18, 25]', u'(25, 35]', u'(35, 60]', u'(60, 100]'], dtype='object')

In [110]:
pd.value_counts(cats) # 返回每个区间的元素数量

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [111]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # 默认左开右闭区间，设置right=False左闭右开

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, object): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [112]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
group_names

['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [113]:
pd.cut(ages, bins, labels=group_names) # 设置面元的名称，将labels选项设置为一个列表或数组

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [114]:
data = np.random.rand(20) # 如果向cut函数传入的是面元的数量而不是区间，会根据数据的最小值、最大值计算等长面元
data

array([ 0.76268593,  0.42258196,  0.20279806,  0.04898332,  0.9823432 ,
        0.00885837,  0.88039994,  0.19566657,  0.89011536,  0.83966726,
        0.82988615,  0.89906255,  0.04312549,  0.04980479,  0.58188674,
        0.94120214,  0.96394591,  0.79443576,  0.22080414,  0.44705843])

In [115]:
pd.cut(data, 4, precision=2) # 第二个参数为整数，则将区间平均切分后进行封箱；precision参数，设置封箱时区间边界小数点后保留的位数

[(0.74, 0.98], (0.25, 0.5], (0.0079, 0.25], (0.0079, 0.25], (0.74, 0.98], ..., (0.74, 0.98], (0.74, 0.98], (0.74, 0.98], (0.0079, 0.25], (0.25, 0.5]]
Length: 20
Categories (4, object): [(0.0079, 0.25] < (0.25, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

#### qcut函数

In [116]:
data = np.random.randn(1000)

In [117]:
cats = pd.qcut(data, 4) # 根据样本分位数对数据进行面元划分（每个面元内元素数量基本相等）
cats

[(0.696, 3.0103], (-0.718, -0.0102], (0.696, 3.0103], (-0.0102, 0.696], [-3, -0.718], ..., (-0.718, -0.0102], (-0.0102, 0.696], [-3, -0.718], (-0.718, -0.0102], (-0.0102, 0.696]]
Length: 1000
Categories (4, object): [[-3, -0.718] < (-0.718, -0.0102] < (-0.0102, 0.696] < (0.696, 3.0103]]

In [118]:
pd.value_counts(cats) # 每个面元内元素数量基本相等，cut是区间长度相等，qcut是区间内元素个数相等

(0.696, 3.0103]      250
(-0.0102, 0.696]     250
(-0.718, -0.0102]    250
[-3, -0.718]         250
dtype: int64

In [119]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]) # 第二个参数，自定义分位数，列表必须以0开头，以1结尾

[(1.329, 3.0103], (-1.351, -0.0102], (1.329, 3.0103], (-0.0102, 1.329], (-1.351, -0.0102], ..., (-1.351, -0.0102], (-0.0102, 1.329], (-1.351, -0.0102], (-1.351, -0.0102], (-0.0102, 1.329]]
Length: 1000
Categories (4, object): [[-3, -1.351] < (-1.351, -0.0102] < (-0.0102, 1.329] < (1.329, 3.0103]]

### 检测和过滤异常值

In [120]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [121]:
col = data[3]
col[np.abs(col) > 3] # 找出某列中绝对值大小超过3的值（布尔型索引）

97     3.927528
305   -3.399312
400   -3.745356
Name: 3, dtype: float64

In [122]:
data[(np.abs(data) > 3).any(1)] # 找出含有绝对值大小超过3的值的行

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [123]:
data[np.abs(data) > 3] = np.sign(data) * 3 # 将DataFrame中的值限制在-3到3以内（超过3的值设置为3）
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 排列和随机采样

#### 随机重排序
numpy.random.permutation函数，可以产生一个表示新顺序的整数数组

In [124]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [125]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [126]:
df.ix[sampler]

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [127]:
df.take(sampler) # 基于ix的索引操作或take函数，进行随机重排序

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


#### 选取随机子集（不重复抽样）
从permutation返回的数组中切下前k个元素，其中k为期望的子集大小

In [128]:
df.take(np.random.permutation(len(df))[:3]) # len函数返回df的行数，不重复抽样3行

,0,1,2,3
1,4,5,6,7
3,12,13,14,15
4,16,17,18,19


#### 选取样本（重复抽样）
numpy.random.randint函数，产生一组随机整数

In [129]:
bag = np.array([5, 7, -1, 6, 4])
bag

array([ 5,  7, -1,  6,  4])

In [130]:
sampler = np.random.randint(0, len(bag), size=10)
sampler

array([4, 4, 2, 2, 2, 0, 3, 0, 4, 1])

In [131]:
draws = bag.take(sampler)
draws

array([ 4,  4, -1, -1, -1,  5,  6,  5,  4,  7])

### 计算指标/哑变量
get_dummies函数，DataFrame的某一列中含有k个不同的值，可以派生出一个k列矩阵或DataFrame（取值全为1和0）

#### 每行只属于一个分类

In [132]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [133]:
pd.get_dummies(df['key']) # 参数是要计算哑变量的列

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [134]:
dummies = pd.get_dummies(df['key'], prefix='key') # 为生成的哑变量矩阵的列名添加前缀
dummies

,key_a,key_b,key_c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


In [135]:
df_with_dummy = df[['data1']].join(dummies) # 将生成的哑变量矩阵添加至原DataFrame
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


#### 每行同属于多个分类

In [136]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('./data/ch02/movies.dat', sep='::', header=None,
                        names=mnames, engine='python') # 读入ch02的MovieLens 1M数据集
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [137]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres # 获取所有出现过的genres，并去除重复值

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [138]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies.head() # 生成空的数据框

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
dummies.head() # 生成稀疏矩阵

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [140]:
movies_windic = movies.join(dummies.add_prefix('Genre_')) # df.add_prefix()，给df的所有列名添加前缀；同理还有df.add_suffix()
movies_windic.ix[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

##### 离散化与哑变量的结合

In [141]:
values = np.random.rand(10)
values

array([ 0.75603383,  0.90830844,  0.96588737,  0.17373658,  0.87592824,
        0.75415641,  0.163486  ,  0.23784062,  0.85564381,  0.58743194])

In [142]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins)) # 将连续数据离散化，进而转化为哑变量矩阵

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,1.0,0.0
6,1.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,1.0,0.0,0.0


## 字符串操作

### 字符串对象方法
+ count  
返回字串在字符串中的出现次数（非重叠）
+ endswith、startswitch  
如果字符串以某个后缀结尾（以某个前缀开头），则返回True
+ join  
将字符串用作连接其它字符串序列的分隔符
+ index  
如果在字符串中找到子串，则返回字串第一个字符所在的位置。如果没有找到，则引发ValueError
+ find  
如果在字符串中找到子串，则返回第一个发现的字串的第一个字符所在的位置。如果没有找到，则返回-1
+ rfind  
如果在字符串中找到子串，则返回最后一个发现的子串的第一个字符所在的位置。如果没有找到，则返回-1
+ replace  
用另一个字符串替换指定子串
+ strip、rstrip、lstrip  
去除空白符（包括换行符）
+ split  
通过指定的分隔符将字符串拆分为一组子串
+ lower、upper  
分别将字母字符转换为小写或大写
+ ljust、rjust  
用空格（或其他字符）填充字符串的空白侧以返回符合最低宽度的字符串

In [143]:
val = 'a,b,  guido'
val

'a,b,  guido'

In [144]:
val.split(',') # 以逗号分隔的字符串，用split拆分成数段，用列表组装

['a', 'b', '  guido']

In [145]:
pieces = [x.strip() for x in val.split(',')] # split与strip结合，修剪空白符（包括换行符）
pieces

['a', 'b', 'guido']

In [146]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [147]:
'::'.join(pieces) # 将列表中的子字符串以双冒号分隔符的形式连接起来，返回新的字符串，以上两种方式等价

'a::b::guido'

In [148]:
'guido' in val # 子串定位，返回True或False

True

In [149]:
val.index(',') # 子串定位

1

In [150]:
val.find(':') # 子串定位，如果找不到字符串，find返回-1

-1

In [151]:
val.count(',') # 返回指定子串的出现次数

2

In [152]:
val.replace(',', '::') # 用第二个参数替换为第一个参数

'a::b::  guido'

In [153]:
val.replace(',', '') # 第二个参数如果传入空字符串可用于删除模式

'ab  guido'

### 正则表达式
正则表达式（regex）提供了一种灵活的在文本中搜索或匹配字符串模式的方式。
Python内置的re模块负责对字符串应用正则表达式。re模块的函数可以分为三大类：模式匹配、替换以及拆分。

+ findall、finditer  
返回字符串中所有的非重叠匹配模式。findall返回的是由所有模式组成的列表，而finditer则通过一个迭代器逐个返回
+ match  
从字符串起始位置匹配模式，还可以对模式各部分进行分组。如果匹配到模式，则返回一个匹配项对象，否则返回None
+ search  
扫描整个字符串以匹配模式。如果找到则返回一个匹配项对象。跟match不同，其匹配项可以位于字符串的任意位置，而不仅仅是起始处
+ split  
根据找到的模式将字符串拆分为数段
+ sub、subn  
将字符串中所有的（sub）或前n个（subn）模式替换为指定表达式。在替换字符串中可以通过\1、\2等符号表示各分组项

#### re模块中的re对象

In [154]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text) # 描述一个或多个空白符的regex是\s+，调用re时，正则表达式会先被编译，然后再在text上调用其split方法

['foo', 'bar', 'baz', 'qux']

#### re.compile编译regex对象

In [155]:
regex = re.compile('\s+') # 用re.compile自己编译regex以得到一个可重用regex对象
regex.split(text)

['foo', 'bar', 'baz', 'qux']

##### 模式匹配

In [156]:
regex.findall(text) # 得到符合regex的所有匹配项，使用findall方法

['    ', '\t ', '  \t']

In [157]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}' # 能够识别电子邮件地址的正则表达式

regex = re.compile(pattern, flags=re.IGNORECASE) # # re.IGNORECASE的作用是使正则表达式对大小写不敏感

In [158]:
regex.findall(text) # findall，返回字符串中所有的匹配项

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [159]:
m = regex.search(text) # search，返回第一个匹配项
m

In [160]:
text[m.start():m.end()] # 以特殊的匹配项对象形式返回，告诉我们模式在原字符串中的起始和结束位置

'dave@google.com'

In [161]:
print(regex.match(text)) # match，匹配出现在字符串开头的模式，未匹配到则返回None，匹配到则返回特殊的匹配项对象

None


##### 替换

In [162]:
print(regex.sub('REDACTED', text)) # sub，将匹配到的所有模式替换为指定字符串，并返回所得到的新字符串

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



##### 拆分

In [163]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})' # 将电邮地址分为3部分：用户名、域名、域后缀，将待分段的模式各部分用圆括号包起来
regex = re.compile(pattern, flags=re.IGNORECASE)

In [164]:
m = regex.match('wesm@bright.net')
m.groups() # 由上述正则表达式所产生的匹配项对象，可以通过其groups方法返回一个由模式各段组成的元组

('wesm', 'bright', 'net')

In [165]:
regex.findall(text) # 对于带有拆分功能的模式，findall会返回一个元组列表

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [166]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text)) # sub通过诸如\1，\2之类的特殊符号访问各匹配项中的分组

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [167]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

In [168]:
m = regex.match('wesm@bright.net')
m.groupdict() # 通过groupdict方法返回一个带有分组名称的字典

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

### pandas中矢量化的字符串函数
+ cat  
实现元素级的字符串连接操作，可指定分隔符
+ contains  
返回表示各字符串是否含有指定模式的布尔型数组
+ count  
模式的出现次数
+ endswith、startswitch  
相当于对各个元素执行x.endswith(pattern)或x.startswith(patter)
+ findall  
计算各字符串的模式列表
+ get  
获取各元素的第i个字符
+ join  
根据指定的分隔符将Series中各元素的字符串连接起来
+ len  
计算各字符串的长度
+ lower、upper  
转换大小写。相当于对各个元素执行x.lower()或x.upper()
+ match  
根据指定的正则表达式对各个元素执行re.match
+ pad  
在字符串的左边、右边或左右两边添加空白符
+ center  
相当于pad(side='both')
+ repeat  
重复值。例如，s.str.repeat(3)相当于对各个字符串执行x*3
+ replace  
用指定字符串替换找到的字符
+ slice  
对Series中的各个字符串进行子串截取
+ split  
根据分隔符或正则表达式对字符串进行拆分
+ strip、rstrip、lstrip  
去除空白符，包括换行符。相当于对各个元素执行x.strip()、x.rstrip()、x.lstrip()

In [169]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [170]:
data.isnull() # 定位缺失值

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

#### 通过Series的str属性，可以访问能够跳过NA值的字符串操作方法

In [171]:
data.str.contains('gmail') # 检查各个电邮地址是否含有“gmail”

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [172]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [173]:
data.str.findall(pattern, flags=re.IGNORECASE) # findall，寻找匹配pattern的所有模式

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

#### 矢量化的元素获取操作

In [174]:
matches = data.str.match(pattern, flags=re.IGNORECASE) 
matches

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

##### str.get

In [175]:
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

##### 在str属性上使用索引

In [176]:
matches.str[0]

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

#### 对字符串进行子串截取

In [177]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

## Example: USDA Food Database

In [ ]:
import json
db = json.load(open('ch07/foods-2011-10-03.json'))
len(db)

In [ ]:
db[0].keys()

In [ ]:
db[0]['nutrients'][0]

In [ ]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

In [ ]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [ ]:
info[:5]

In [ ]:
info

In [ ]:
pd.value_counts(info.group)[:10]

In [ ]:
nutrients = []

for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

In [ ]:
nutrients

In [ ]:
nutrients.duplicated().sum()

In [ ]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {'description' : 'food',
               'group'       : 'fgroup'}
info = info.rename(columns=col_mapping, copy=False)
info

In [ ]:
col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)
nutrients

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [ ]:
ndata

In [ ]:
ndata.ix[30000]

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].sort_values().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

# make the food a little smaller
max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.ix['Amino Acids']['food']